# Pre-Processing: Dataset Creation

## IMPORTANT Note for Reproducibility:

This code will scrape the **LATEST Census data** on CensusReporter. Our paper only uses 2019 Census data. To use the dataset used for our paper, click on this link: https://github.com/JimiLab/LocalifyMusicEventData. 

**Only use this notebook if you are using the LATEST Census data.**

NOTE: Due to the nature of the procedure in this notebook, we advise that this notebook be run ONE AT A TIME (NOT all at once!)

In [1]:
import pandas as pd
import censusdata
import requests
import webbrowser
import os
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import numpy as np
import re
import time
from tqdm import tqdm, trange
import math
from bs4 import BeautifulSoup as bs
import math
import unidecode
import json
from lxml import etree

import mysql.connector
from time import sleep

### Read CSV Files and Preprocess them

We are only looking at city-level data here. Any non-city-level data (such as county-level data) are filtered out. Any data based in US territories such as Puerto Rico are also filtered out.

In [2]:
# Below excel spreadsheet downloaded from https://www2.census.gov/programs-surveys/metro-micro/geographies/reference-files/2020/delineation-files/list2_2020.xls
df = pd.read_excel('data/list2_2020.xls') 
df = df[2:-4]
df.rename(columns={'Table with row headers in column A and column headers in row 3':'CBSA Code', 'Unnamed: 1':'CBSA Title', 'Unnamed: 2':'Metropolitan/Micropolitan Statistical Area', 'Unnamed: 3':'Principal City Name', 'Unnamed: 4':'FIPS State Code', 'Unnamed: 5':'FIPS Place Code'}, inplace=True)
df = df.set_index('CBSA Code')

In [3]:
# Exclude Puerto Rico (which has state code 72)
df = df[df['FIPS State Code'] != '72']

In [4]:
# Below excel spreadsheet downloaded from https://gist.github.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53
states = pd.read_csv('data/us-state-ansi-fips.csv', delimiter=',') 
states.columns=['stname', 'st', 'stusps']
states.index = states['st']

In [5]:
state_abbrv = []
for i in range(len(df)):
    state_abbrv.append(states['stusps'][int(df['FIPS State Code'][i])])

df['Principal City Name'] = df['Principal City Name'] 
df['State'] = state_abbrv
df["State"] = df["State"].str.strip()

In [6]:
state_name = []
for i in range(len(df)):
    state_name.append(states['stname'][int(df['FIPS State Code'][i])])

df['State Name'] = state_name

In [7]:
# Get rid of actual counties that are listed as a "Principal City":
df = df[df['Principal City Name'].str.contains('County') == False]

# Get rid of "Nashville-Davidson metropolitan government"
df = df[~df['Principal City Name'].str.contains('government')]

# "Butte-Silver Bow" is not a city
df = df[~df['Principal City Name'].str.contains('Butte')]

# "Lexington-Fayette" is not a city, but a county
df = df[~(df['Principal City Name'] == 'Lexington-Fayette')]

### Get Corresponding Latitude, Longitude Coordinates for each Area

Note that an area can have multiple cities/counties. 

Some of the principal cities below may require manual manipulation (as the region names may need to be cleaned up).

WARNING: This cell can take just over 10 minutes to run. Try to reduce the number of times you run this cell below!!

In [26]:
# Get longitudes and latitudes of each individual city. An area can have multiple cities. 
latitudes = []
longitudes = []

start = time.time()

gn = Nominatim(user_agent="name")

for i in tqdm(range(len(df))):
    city = df['Principal City Name'].iloc[i]
    state = df['State'].iloc[i]
    loc = gn.geocode(city + ', ' + state)
    latitudes.append(loc.latitude if loc is not None else np.nan)
    longitudes.append(loc.longitude if loc is not None else np.nan)

print('Time elapsed:', time.time() - start, 'seconds')

100%|██████████| 1246/1246 [10:30<00:00,  1.98it/s]

Time elapsed: 630.5243608951569 seconds


In [27]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

Below areas require manual manipulation
(Paper: Put on Appendix)

In [28]:
# "Indianapolis city (balance), IN" -> "Indianapolis, IN"
a = gn.geocode('Indianapolis, IN')
df.at['26900', 'Latitude'].iloc[2] = a.latitude
df.at['26900', 'Longitude'].iloc[2] =  a.longitude

/Users/michaelzhou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [30]:
# "Urban Honolulu, HI" -> "Honolulu, HI"
a = gn.geocode('Honolulu, HI')
df.at['46520', 'Latitude'] = a.latitude
df.at['46520', 'Longitude'] = a.longitude

In [31]:
# "San Luis Obispo, CA" coordinates corrected via latlong.net
df.at['42020', 'Latitude'].iloc[1] = 35.2827524
df.at['42020', 'Longitude'].iloc[1] = -120.6596156

In [32]:
# "Napa, CA" coordinates corrected via latlong.net
df.at['34900', 'Latitude'] = 38.297539
df.at['34900', 'Longitude'] = -122.286865

In [33]:
# Geocode confuses "Middletown, NY" with the "Midtown NY", so we have to change manually from latlong.net
df.at['39100', 'Latitude'].iloc[3] = 41.450123 
df.at['39100', 'Longitude'].iloc[3] = -74.429253

In [34]:
# "Grenada, MS" coordinates corrected via latlong.net
df.at['24980', 'Latitude'] = 33.7817513
df.at['24980', 'Longitude'] = -89.81306

In [35]:
# Inspect missing location coordinate values
assert(len(df[df['Latitude'].isnull().values]) == 0)
assert(len(df[df['Longitude'].isnull().values]) == 0)

### Retrieve Socioeconomic Indicators

We will retrieve different socioeconomic indicators by scraping four different websites - CensusReporter, DataUSA, Census Quickfacts, and AreaVibes. 

#### CensusReporter

Retrieve 2019 population, population density, median age, percent of population per age group, per capita income, median household income, education-related rates, mean travel time, transportation rates (walking, public transit, biking), poverty rate, median property value, migration rate since previous year, percent of population per race.

Example query: https://censusreporter.org/profiles/16000US2002900-atchison-ks/

In [38]:
# Find relevant script from CensusReporter
def find_relevant_script(soup):
    script_tags = soup.findAll('script')
    
    for script in script_tags:
        if 'profileData' in str(script):
            return str(script)

    return None

WARNING: The following chunk of code can take 15-20 minutes to run:

In [49]:
# Population/population density:
population_2019 = []
population_density = []

# Median Age:
median_age = []

# Age diversity:
population_under_18 = []
population_18_to_64 = []
population_over_65 = []

population_0_to_9 = []
population_10_to_19 = []
population_20_to_29 = []
population_30_to_39 = []
population_40_to_49 = []
population_50_to_59 = []
population_60_to_69 = []
population_70_to_79 = []
population_over_80 = []

proportion_under_18 = []
proportion_18_to_64 = []
proportion_over_65 = []

proportion_0_to_9 = []
proportion_10_to_19 = []
proportion_20_to_29 = []
proportion_30_to_39 = []
proportion_40_to_49 = []
proportion_50_to_59 = []
proportion_60_to_69 = []
proportion_70_to_79 = []
proportion_over_80 = []

# Income:
per_capita_income = []
median_household_income = []

# Education:
highschool_or_higher = []
bachelors_or_higher = []
postgrad = []

# Transportation:
mean_travel_time = []
public_transit = []
bicycle = []
walkability = []

# Poverty rate:
poverty_rate = []

# Median value of owner-occupied housing units:
median_property_value = []

# Percentage moved since previous year:
migration_rate_since_previous_year = []

# Race populations:
population_white = []
population_black = []
population_native = []
population_asian = []
population_islander = []
population_other = []
population_two_or_more = []
population_hispanic = []

for i in tqdm(range(len(df))):
    row = df.iloc[i]
    
    url = 'https://censusreporter.org/profiles/16000US' + f"{int(row['FIPS State Code']):02d}" + f"{int(row['FIPS Place Code']):05d}"
    response = requests.get(url)
        
    soup = bs(response.text)
    js_script = find_relevant_script(soup)
    data_string = re.search('(?<=profileData = )(.*)(?=;)', js_script).group(0)
    
    profile_data = json.loads(data_string)
    
    ### WARNING: Asserting this statement will fail, as CensusReporter is no longer using ACS-2019 data.
    # assert('2019' in profile_data['geography']['census_release'])

    # Population/population density:
    population_2019.append(profile_data['geography']['this']['total_population'])
    population_density.append(profile_data['geo_metadata']['population_density'])

    # Median Age:
    median_age.append(profile_data['demographics']['age']['median_age']['total']['values']['this'])

    # Age diversity:
    population_under_18.append(profile_data['demographics']['age']['distribution_by_category']['percent_under_18']['numerators']['this'])
    population_18_to_64.append(profile_data['demographics']['age']['distribution_by_category']['percent_18_to_64']['numerators']['this'])
    population_over_65.append(profile_data['demographics']['age']['distribution_by_category']['percent_over_65']['numerators']['this'])
    
    population_0_to_9.append(profile_data['demographics']['age']['distribution_by_decade']['total']['0-9']['numerators']['this'])
    population_10_to_19.append(profile_data['demographics']['age']['distribution_by_decade']['total']['10-19']['numerators']['this'])
    population_20_to_29.append(profile_data['demographics']['age']['distribution_by_decade']['total']['20-29']['numerators']['this'])
    population_30_to_39.append(profile_data['demographics']['age']['distribution_by_decade']['total']['30-39']['numerators']['this'])
    population_40_to_49.append(profile_data['demographics']['age']['distribution_by_decade']['total']['40-49']['numerators']['this'])
    population_50_to_59.append(profile_data['demographics']['age']['distribution_by_decade']['total']['50-59']['numerators']['this'])
    population_60_to_69.append(profile_data['demographics']['age']['distribution_by_decade']['total']['60-69']['numerators']['this'])
    population_70_to_79.append(profile_data['demographics']['age']['distribution_by_decade']['total']['70-79']['numerators']['this'])
    population_over_80.append(profile_data['demographics']['age']['distribution_by_decade']['total']['80+']['numerators']['this'])
    
    proportion_under_18.append(profile_data['demographics']['age']['distribution_by_category']['percent_under_18']['values']['this'])
    proportion_18_to_64.append(profile_data['demographics']['age']['distribution_by_category']['percent_18_to_64']['values']['this'])
    proportion_over_65.append(profile_data['demographics']['age']['distribution_by_category']['percent_over_65']['values']['this'])
    
    proportion_0_to_9.append(profile_data['demographics']['age']['distribution_by_decade']['total']['0-9']['values']['this'])
    proportion_10_to_19.append(profile_data['demographics']['age']['distribution_by_decade']['total']['10-19']['values']['this'])
    proportion_20_to_29.append(profile_data['demographics']['age']['distribution_by_decade']['total']['20-29']['values']['this'])
    proportion_30_to_39.append(profile_data['demographics']['age']['distribution_by_decade']['total']['30-39']['values']['this'])
    proportion_40_to_49.append(profile_data['demographics']['age']['distribution_by_decade']['total']['40-49']['values']['this'])
    proportion_50_to_59.append(profile_data['demographics']['age']['distribution_by_decade']['total']['50-59']['values']['this'])
    proportion_60_to_69.append(profile_data['demographics']['age']['distribution_by_decade']['total']['60-69']['values']['this'])
    proportion_70_to_79.append(profile_data['demographics']['age']['distribution_by_decade']['total']['70-79']['values']['this'])
    proportion_over_80.append(profile_data['demographics']['age']['distribution_by_decade']['total']['80+']['values']['this'])
    
    
    # Income:
    per_capita_income.append(profile_data['economics']['income']['per_capita_income_in_the_last_12_months']['values']['this'])
    median_household_income.append(profile_data['economics']['income']['median_household_income']['values']['this'])

    # Education:
    highschool_or_higher.append(profile_data['social']['educational_attainment']['percent_high_school_grad_or_higher']['values']['this'])
    bachelors_or_higher.append(profile_data['social']['educational_attainment']['percent_bachelor_degree_or_higher']['values']['this'])
    postgrad.append(profile_data['social']['educational_attainment_distribution']['post_grad_degree']['values']['this'])

    # Transportation:
    mean_travel_time.append(profile_data['economics']['employment']['mean_travel_time']['values']['this'])
    public_transit.append(profile_data['economics']['employment']['transportation_distribution']['public_transit']['values']['this'])
    bicycle.append(profile_data['economics']['employment']['transportation_distribution']['Bicycle']['values']['this'])
    walkability.append(profile_data['economics']['employment']['transportation_distribution']['walked']['values']['this'])

    # Poverty rate:
    poverty_rate.append(profile_data['economics']['poverty']['percent_below_poverty_line']['values']['this'])

    # Median value of owner-occupied housing units:
    median_property_value.append(profile_data['housing']['ownership']['median_value']['values']['this'])

    # Percentage moved since previous year:
    migration_rate_since_previous_year.append(profile_data['housing']['migration']['moved_since_previous_year']['values']['this'])

    # Race populations:
    population_white.append(profile_data['demographics']['race']['percent_white']['numerators']['this'])
    population_black.append(profile_data['demographics']['race']['percent_black']['numerators']['this'])
    population_native.append(profile_data['demographics']['race']['percent_native']['numerators']['this'])
    population_asian.append(profile_data['demographics']['race']['percent_asian']['numerators']['this'])
    population_islander.append(profile_data['demographics']['race']['percent_islander']['numerators']['this'])
    population_other.append(profile_data['demographics']['race']['percent_other']['numerators']['this'])
    population_two_or_more.append(profile_data['demographics']['race']['percent_two_or_more']['numerators']['this'])
    population_hispanic.append(profile_data['demographics']['race']['percent_hispanic']['numerators']['this'])

# Population/population density:
df['Population'] = population_2019
df['Population Density'] = population_density

# Median Age:
df['Median Age'] = median_age

# Age diversity:
df['Population Under 18'] = population_under_18
df['Population 18-64'] = population_18_to_64
df['Population Over 65'] = population_over_65

df['Population 0-9'] = population_0_to_9
df['Population 10-19'] = population_10_to_19
df['Population 20-29'] = population_20_to_29
df['Population 30-39'] = population_30_to_39
df['Population 40-49'] = population_40_to_49
df['Population 50-59'] = population_50_to_59
df['Population 60-69'] = population_60_to_69
df['Population 70-79'] = population_70_to_79
df['Population Over 80'] = population_over_80

df['Percent Under 18'] = proportion_under_18 
df['Percent 18-64'] = proportion_18_to_64 
df['Percent Over 65'] = proportion_over_65

df['Percent 0-9'] = proportion_0_to_9
df['Percent 10-19'] = proportion_10_to_19
df['Percent 20-29'] = proportion_20_to_29
df['Percent 30-39'] = proportion_30_to_39
df['Percent 40-49'] = proportion_40_to_49
df['Percent 50-59'] = proportion_50_to_59
df['Percent 60-69'] = proportion_60_to_69
df['Percent 70-79'] = proportion_70_to_79
df['Percent Over 80'] = proportion_over_80

# Income:
df['Per Capita Income'] = per_capita_income
df['Median Household Income'] = median_household_income

# Education:
df['High School Or Higher'] = highschool_or_higher
df['Bachelor Or Higher'] = bachelors_or_higher
df['Postgrad Degree'] = postgrad

# Transportation:
df['Mean Travel Time'] = mean_travel_time
df['Public Transit'] = public_transit
df['Bicycle'] = bicycle
df['Walkability'] = walkability

# Poverty rate:
df['Poverty Rate'] = poverty_rate

# Median value of owner-occupied housing units:
df['Median Property Value'] = median_property_value

# Percentage moved since previous year:
df['Migration Rate Since Previous Year'] = migration_rate_since_previous_year

# Race populations:
df['Population White'] = population_white
df['Population Black'] = population_black
df['Population Native'] = population_native
df['Population Asian'] = population_asian
df['Population Islander'] = population_islander
df['Population Other'] = population_other
df['Population Two+'] = population_two_or_more
df['Population Hispanic'] = population_hispanic

100%|██████████| 1246/1246 [18:24<00:00,  1.13it/s]


#### DataUSA

Retrieve employed population and calculate employment rate. 

Example query: https://datausa.io/profile/geo/middletown-ny

In [62]:
# Find index given DataUSA HTML tag list
def find_index(tag_list, field):
    i = 0
    for profile in tag_list:
        if field in profile:
            return i
        i+=1
    return -1

In [63]:
# Get value from DataUSA
def get_value(tag_list, field):
    i = find_index(tag_list, field)
    return tag_list[i+1].string 

WARNING: The following chunk of code may take about 10-20 minutes to run (unless the web requests are cached):

In [72]:
employed_population = []

m = {'K': 3, 'M': 6, 'B': 9, 'T': 12} # for converting strings like '2.69M' to 2690000

start = time.time()

start_index = 0

for i in tqdm(range(start_index, len(df)), initial=start_index):
    row = df.iloc[i]
    
    city = row['Principal City Name']
    state = row['State']
    city_ticker = city.lower().replace(',', '').replace('.', '').replace('/','').replace(' ', '-').replace("'",'').replace('(', '').replace(')', '')
    city_ticker = unidecode.unidecode(city_ticker) # remove accents (e.g. replace 'ñ' with 'n')
    
    results = requests.get('https://datausa.io/profile/geo/' + city_ticker + '-' + state.lower().strip())
    soup = bs(results.text)
    tag_list = soup.findAll(True, {'class':['stat-title', 'stat-value', 'stat-subtitle']})
    
    ep = get_value(tag_list, '2019 Employed Population')
    employed_population.append(math.nan if ep[-2].isalpha() else int(ep.replace(',','')) if not ep[-1].isalpha() else int(float(ep[:-1]) * 10**m[ep[-1]]))
    
    # Wait for some time before the next request 
    sleep(0.1)

print('Time elapsed:', time.time() - start, 'seconds')

df['Employed Population'] = employed_population
df['Employment Rate'] = df['Employed Population'] / df['Population']

100%|██████████| 1246/1246 [09:31<00:00,  2.18it/s]

Time elapsed: 571.6628119945526 seconds


#### Census QuickFacts

Retrieve population from 4/1/2010, median gross rent (2015-2019), median owner costs with mortgage (2015-2019), median owner costs without mortgage (2015-2019), owner-occupied housing unit rate (2015-2019).

Example queries: 
- https://www.census.gov/quickfacts/fact/table/milwaukeecitywisconsin
- https://www.census.gov/quickfacts/fact/table/senecafallscdpnewyork 

Note that we query at most 6 cities at a time (to save time). If there is no data available for a certain city, we ignore it. 

So a sample query from our implementation would be similar to this:
https://www.census.gov/quickfacts/fact/table/annapoliscitymaryland,amsterdamcitynewyork,andrewscitytexas,angolacityindiana,annarborcitymichigan,annistoncityalabama

In [73]:
# Get a certain field value from Census Quickfacts
def get_values(soup, field, num):
    text = ''
    td_tags = soup.findAll('td')
    i = 0
    for body in td_tags:
        if field in str(body):
            text = body
            break
        i+=1
    
    vals = []
    for j in range(1, num+1):
        try: 
            vals.append(float(td_tags[i+j].attrs.get("data-value", None)))
        except:
            continue
    
    return vals

In [74]:
# Get Census population recorded on April 1, 2010
def get_2010_population(soup, num):
    return get_values(soup, 'Population, Census, April 1, 2010', num)

# Get Median Gross Rent from 2015-2019
def get_median_gross_rent(soup, num):
    return get_values(soup, 'Median gross rent, 2015-2019', num)

# Get Median selected monthly owner costs -with a mortgage, 2015-2019
def get_median_owner_cost_with_mortgage(soup, num):
    return get_values(soup, 'Median selected monthly owner costs -with a mortgage, 2015-2019', num)

# Get Median selected monthly owner costs -without a mortgage, 2015-2019
def get_median_owner_cost_without_mortgage(soup, num):
    return get_values(soup, 'Median selected monthly owner costs -without a mortgage, 2015-2019', num)

# Get Owner-occupied housing unit rate, 2015-2019
def get_owner_occupied_housing_unit_rate(soup, num):
    return get_values(soup, 'Owner-occupied housing unit rate, 2015-2019', num)

WARNING: The following chunk of code will take over an hour to run!!!

In [75]:
# Get the associated string to the city for the URL (e.g. "Milwaukee, WI" is "city" for "milwaukeecitywisconsin")
# Usually, the principal city will be followed by "city" as a suffix before the state; 
# however, there are some exceptions in which the suffix is actually "town", "village", "borough", "cdp", etc.
# The code below specifies these exceptions:
def get_city_str(city, state):
    city_str = 'city'

    if 'balance' in city_ticker or (city == 'Carson City' and state == 'Nevada') or (city == 'Princeton' and state == 'New Jersey'):
        city_str = ''

    elif city == 'Anchorage' and state == 'Alaska': # Anchorage, AL only shows up as 'Anchorage municipality'
        city_str = 'municipality'

    elif (state == 'Pennsylvania' and (city == 'Berwick' 
                                       or city == 'Chambersburg' 
                                       or city == 'Waynesboro' 
                                       or city == 'East Stroudsburg' 
                                       or city == 'Gettysburg'
                                       or city == 'Carlisle' 
                                       or city == 'Huntingdon' 
                                       or city == 'Indiana'  
                                       or city == 'Lewisburg'
                                      or city == 'Lewistown'
                                      or city == 'Somerset'
                                      or city == 'Sayre'
                                      or city == 'Selinsgrove'
                                      or city == 'State College'
                                      or city == 'Hanover')):
        city_str = 'borough'
    
    elif city == 'Juneau' and state == 'Alaska':
        city_str = 'cityandborough'
    
    elif city == 'Norwich' and state == 'Connecticut':
        city_str = 'townnewlondoncounty'
    
    elif (city == 'Stratford' and state == 'Connecticut'):
        city_str = 'town' + 'fairfieldcounty'

    elif (city == 'Bolingbrook' and state == 'Illinois'
          or city == 'Hoffman Estates' and state == 'Illinois'
          or city == 'Schaumburg' and state == 'Illinois'
          or city == 'Skokie' and state == 'Illinois'
          or city == 'Fredonia' and state == 'New York'
          or city == 'Malone' and state == 'New York'
          or city == 'Massena' and state == 'New York'
         or city == 'Pinehurst' and state == 'North Carolina'
         or city == 'Ruidoso' and state == 'New Mexico'
         or city == 'Weston' and state == 'Wisconsin'):
        city_str = 'village'

    elif (city == 'Hammonton' and state == 'New Jersey'
          or city == 'Big Stone Gap' and state == 'Virginia'
          or city == 'Blacksburg' and state == 'Virginia'
          or city == 'Christiansburg' and state == 'Virginia'
          or city == 'Bloomsburg' and state == 'Pennsylvania'
          or city == 'Boone' and state == 'North Carolina'
          or city == 'Breckenridge' and state == 'Colorado'
          or city == 'Chapel Hill' and state == 'North Carolina'
          or city == 'Easton' and state == 'Maryland'
          or city == 'Forest City' and state == 'North Carolina'
          or city == 'Greeneville' and state == 'Tennessee'
          or city == 'Bluffton' and state == 'South Carolina'
          or city == 'Hilton Head Island' and state == 'South Carolina'
          or city == 'Jackson' and state == 'Wyoming'
          or city == 'Kill Devil Hills' and state == 'North Carolina'
          or city == 'Jupiter' and state == 'Florida'
          or city == 'Morehead City' and state == 'North Carolina'
          or city == 'Prescott Valley' and state == 'Arizona'
          or city == 'Cary' and state == 'North Carolina'
         or city == 'Payson' and state == 'Arizona'
         or city == 'Southern Pines' and state == 'North Carolina'
         or city == 'Silver City' and state == 'New Mexico'
         or city == 'Taos' and state == 'New Mexico'
         or city == 'Truckee' and state == 'California'):
        city_str = 'town'
    
    elif (state == 'Hawaii' 
          or city == 'Columbia' and state == 'Maryland' 
          or city == 'Towson' and state == 'Maryland'
          or city == 'Bennington' and state == 'Vermont'
          or city == 'Silverdale' and state == 'Washington'
          or city == 'Cheektowaga' and state == 'New York'
          or city == 'California' and state == 'Maryland'
          or city == 'Lexington Park' and state == 'Maryland'
          or city == 'Cullowhee' and state == 'North Carolina'
          or city == 'Edwards' and state == 'Colorado'
          or city == 'Fort Knox' and state == 'Kentucky'
          or city == 'Fort Leonard Wood' and state == 'Missouri'
          or city == 'Fort Polk South' and state == 'Louisiana'
          or city == 'Gardnerville Ranchos' and state == 'Nevada'
          or city == 'East Hartford' and state == 'Connecticut'
          or city == 'Homosassa Springs' and state == 'Florida'
          or city == 'The Woodlands' and state == 'Texas'
          or city == 'Paradise' and state == 'Nevada'
          or city == 'Los Alamos' and state == 'New Mexico'
          or city == 'Kendall' and state == 'Florida'
          or city == 'Metairie' and state == 'Louisiana'
          or city == 'Lakewood' and state == 'New Jersey'
          or city == 'Ferry Pass' and state == 'Florida'
          or city == 'The Villages' and state == 'Florida'
          or city == 'Arlington' and state == 'Virginia'
          or city == 'Reston' and state == 'Virginia'
          or city == 'Pahrump' and state == 'Nevada'
         or city == 'Brent' and state == 'Florida'
         or city == 'Woodbury' and state == 'New York'
         or city == 'Seneca Falls' and state == 'New York'
         or city == 'Bethesda' and state == 'Maryland'
         or city == 'Fort Drum' and state == 'New York'
         or city == 'Zapata' and state == 'Texas'):
        city_str = 'cdp'
    
    return city_str

In [76]:
num = 6
df.index = range(len(df))

start = time.time()

start_index = 0

with tqdm(total=len(df), initial=start_index) as pbar:
    for i in range(start_index, len(df), num):
        city_tickers = ''
        for j in range(num):
            
            if i+j >= len(df):
                break
            
            row = df.iloc[i+j]
            city = row['Principal City Name']
            state = row['State Name'].strip()
            
            city_ticker = city.lower().replace(',', '').replace('.', '').replace('/','').replace(' ', '').replace("'",'').replace('(', '').replace(')', '').replace('-','')
            city_ticker = unidecode.unidecode(city_ticker)
            city_str = get_city_str(city, state)
            city_ticker = city_ticker + city_str + state.lower().replace(' ', '')

            city_tickers += city_ticker + ','

        url = 'https://www.census.gov/quickfacts/fact/table/' + city_tickers[:-1]
        response = requests.get(url)

        soup = bs(response.text)
        
        population_2010 = get_2010_population(soup, num)
        median_gross_rent = get_median_gross_rent(soup, num)
        median_owner_cost_with_mortgage = get_median_owner_cost_with_mortgage(soup,num)
        median_owner_cost_without_mortgage = get_median_owner_cost_without_mortgage(soup,num)
        owner_occupied_housing_unit_rate = get_owner_occupied_housing_unit_rate(soup,num)
        
        k = 0
        for j in range(num):
            
            if i+j >= len(df):
                break
            
            row = df.iloc[i+j]
            city = row['Principal City Name']
            state = row['State Name'].strip()
            
            # The following cities do not have any relevant data 
            if (city == 'Cornelia' and state == 'Georgia'
                or city == 'Mount Gay-Shamrock' and state == 'West Virginia'
                or city == 'North Wilkesboro' and state == 'North Carolina'
               or city == 'Point Pleasant' and state == 'West Virginia'
               or city == 'Summerville' and state == 'Georgia'
               or city == 'Vineyard Haven' and state == 'Massachusetts'
               or city == 'Wauchula' and state == 'Florida'
               or city == 'Boardman' and state == 'Ohio'):
                assert(math.isnan(df.at[i+j, 'Population (2010)']))
                assert(math.isnan(df.at[i+j, 'Median Gross Rent']))
                assert(math.isnan(df.at[i+j, 'Median Owner Cost With Mortgage']))
                assert(math.isnan(df.at[i+j, 'Median Owner Cost Without Mortgage']))
                assert(math.isnan(df.at[i+j, 'Owner-Occupied Housing Unit Rate']))
                continue
            
            df.at[i+j, 'Population (2010)'] = population_2010[k] 
            df.at[i+j, 'Median Gross Rent'] = median_gross_rent[k] 
            df.at[i+j, 'Median Owner Cost With Mortgage'] = median_owner_cost_with_mortgage[k] 
            df.at[i+j, 'Median Owner Cost Without Mortgage'] = median_owner_cost_without_mortgage[k] 
            df.at[i+j, 'Owner-Occupied Housing Unit Rate'] = owner_occupied_housing_unit_rate[k]
            
            k += 1
        
        pbar.update(num)

print('Elapsed time:', time.time() - start, 'seconds')

1248it [1:06:20,  3.19s/it]                          

Elapsed time: 3980.339728116989 seconds


In [77]:
assert(len(df[df['Population (2010)'].isnull().values]) == 8)

#### AreaVibes.com

Retrieve the crime rates (number of reported incidents per 100K people): total crime, violent crime, property crime

Example query: https://www.areavibes.com/indianapolis-in/crime/

In [78]:
# Get Crime Rate (number of reported incidents per 100K people) from AreaVibes.com
def get_crime_rate(table_td_tags, crime_type='Total crime'):
    i = 0
    for tag in table_td_tags:
        if crime_type in tag.text:
            break
        i+=1
    
    result = table_td_tags[i+3].text.replace(',','').replace('(estimate)','').strip()
    return math.nan if result=='n/a' else float(result)

WARNING: The code snippet below takes about 7-9 minutes to run:

In [79]:
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    city = row['Principal City Name']
    state = row['State']

    if city.startswith('Indianapolis') or city.startswith('Boise') or city.startswith('Urban H'):
        city = city.lower().replace('city','').replace('urban', '').strip()

    city = city.lower().replace('(balance)','').strip()
    city_ticker = city.replace(' ', '+').lower() + '-' + state.lower().strip()
    city_ticker = unidecode.unidecode(city_ticker)

    url = f'https://www.areavibes.com/{city_ticker}/crime/'

    response = requests.get(url)

    if response.status_code != 200:
        print(i, 'Something Wrong', url)
        continue
    
    soup = bs(response.text)

    table_td_tags = soup.find('table').findAll('td')
    total_crime_rate = get_crime_rate(table_td_tags, 'Total crime')
    violent_crime_rate = get_crime_rate(table_td_tags, 'Violent crime')
    property_crime_rate = get_crime_rate(table_td_tags, 'Property crime')

    df.at[i, 'Total Crime Rate'] = total_crime_rate
    df.at[i, 'Violent Crime Rate'] = violent_crime_rate
    df.at[i, 'Property Crime Rate'] = property_crime_rate
    
    # Wait for some time before the next request 
    sleep(0.1)

100%|██████████| 1246/1246 [09:00<00:00,  2.31it/s]


### Calculate metrics

Diversity Index Formula (Used for Race and Age):

$D = 1 - \frac{\sum{n (n-1)}}{N (N-1)}$

where:
$n$ = Number of individuals of each species/category, 
$N$ = Number of total individuals of all species/categories

Reference: https://www.statisticshowto.com/simpsons-diversity-index/ 

In [80]:
# Race
populations_per_race = df[['Population White', 'Population Black', 'Population Native', 'Population Asian', 'Population Islander', 'Population Other', 'Population Two+', 'Population Hispanic']]
total_populations_race = populations_per_race.sum(axis=1)
df['Race Diversity Index'] = (populations_per_race * (populations_per_race - 1)).sum(axis=1) / (total_populations_race * (total_populations_race - 1))

In [81]:
# Age
populations_per_age = df[['Population 0-9', 'Population 10-19', 'Population 20-29', 'Population 30-39', 'Population 40-49', 'Population 50-59', 'Population 60-69', 'Population 70-79', 'Population Over 80']]
total_populations_age = populations_per_age.sum(axis=1)
df['Age Diversity Index'] = (populations_per_age * (populations_per_age - 1)).sum(axis=1) / (total_populations_age * (total_populations_age - 1))

Define percent of population under 30 years old, 10-29 years old, and 18-29 years old:

In [82]:
df['Percent Under 30'] = df['Percent 0-9'] + df['Percent 10-19'] + df['Percent 20-29']

In [83]:
df['Percent 10-29'] = df['Percent 10-19'] + df['Percent 20-29']
df['Percent 18-29'] = df['Percent Under 30'] - df['Percent Under 18']

Add the percentages taking public transit, biking, and walking together:

In [84]:
df['Public Transit+Bicycle+Walkability'] = df[['Public Transit', 'Bicycle', 'Walkability']].sum(axis=1)

Calculate population growth from 2010 to 2019:

In [85]:
df['10 Year Population Growth'] = df['Population'] - df['Population (2010)']

### Reorganize Columns

In [165]:
df_copy = pd.read_excel('data/list2_2020.xls') # Download from https://www2.census.gov/programs-surveys/metro-micro/geographies/reference-files/2020/delineation-files/list2_2020.xls
df_copy = df_copy[2:-4]
df_copy.rename(columns={'Table with row headers in column A and column headers in row 3':'CBSA Code', 'Unnamed: 1':'CBSA Title', 'Unnamed: 2':'Metropolitan/Micropolitan Statistical Area', 'Unnamed: 3':'Principal City Name', 'Unnamed: 4':'FIPS State Code', 'Unnamed: 5':'FIPS Place Code'}, inplace=True)
df_copy = df_copy.set_index('CBSA Code')

# Exclude Puerto Rico (which has state code 72)
df_copy = df_copy[df_copy['FIPS State Code'] != '72']

# Get rid of actual counties that are listed as a "Principal City":
df_copy = df_copy[df_copy['Principal City Name'].str.contains('County') == False]

# Get rid of "Nashville-Davidson metropolitan government"
df_copy = df_copy[~df_copy['Principal City Name'].str.contains('government')]

# "Butte-Silver Bow" is not a city
df_copy = df_copy[~df_copy['Principal City Name'].str.contains('Butte')]

# "Lexington-Fayette" is not a city, but a county
df_copy = df_copy[~(df_copy['Principal City Name'] == 'Lexington-Fayette')]

df['CBSA Code'] = df_copy.index

In [166]:
cols = ['CBSA Code', 'CBSA Title',
   'Metropolitan/Micropolitan Statistical Area', 'Principal City Name',
   'State', 'State Name', 'FIPS State Code', 'FIPS Place Code', 'Latitude', 'Longitude', 
        
    'Population', 'Population (2010)', '10 Year Population Growth',
    'Population Density', 'Migration Rate Since Previous Year',
        
    'Mean Travel Time', 'Public Transit', 'Bicycle', 'Walkability', 
    'Public Transit+Bicycle+Walkability',
    
    'Per Capita Income', 'Median Household Income', 'Poverty Rate', 'Median Property Value',
    'Employed Population', 'Employment Rate', 'Median Gross Rent',
    'Median Owner Cost With Mortgage', 'Median Owner Cost Without Mortgage',
    'Owner-Occupied Housing Unit Rate',
    
    'Median Age', 'Population 0-9', 'Population 10-19',
   'Population 20-29', 'Population 30-39', 'Population 40-49',
   'Population 50-59', 'Population 60-69', 'Population 70-79',
   'Population Over 80', 'Population Under 18',
   'Population 18-64', 'Population Over 65', 'Percent Under 18',
   'Percent 18-64', 'Percent Over 65', 'Percent 0-9', 'Percent 10-19',
   'Percent 20-29', 'Percent 30-39', 'Percent 40-49', 'Percent 50-59',
   'Percent 60-69', 'Percent 70-79', 'Percent Over 80', 
    'Age Diversity Index', 'Percent Under 30', 'Percent 10-29', 'Percent 18-29',
        
    'High School Or Higher', 'Bachelor Or Higher', 'Postgrad Degree', 
        
    'Population White', 'Population Black', 'Population Native',
    'Population Asian', 'Population Islander', 'Population Other',
    'Population Two+', 'Population Hispanic', 'Race Diversity Index',
        
    'Total Crime Rate', 'Violent Crime Rate', 'Property Crime Rate'
   ]

df = df[cols]
df = df.loc[:,~df.columns.duplicated()]

### Count number of shows per city

Connect Localify MySQL database.

Get a count of shows in each city in the year of 2019.

Make sure that you add the database to your MySQL server using the "localify_2021_12_08_week_bkup.sql" file.

You should also create a 'my.conf' file, whose contents should look something like this:

``
[connector_python]
host = 127.0.0.1
database = database_name
user = your_user
password = your_password
``

In [131]:
conn = mysql.connector.connect(option_files='my.conf') # Use your OWN my.conf file!

start_date = '2019-01-01' # start from 2019
end_date = '2019-12-31' # end in 2019

WARNING: This following code segment can over a minute to run (unless cached)

In [132]:
# Note: This following segment takes about 20 seconds!

# Calculate the number of events listed in a certain city, state. 

# One exception was noted: 
# The 1 event listed in Paradise, NV is a typo in the database - it was listed in "aradise" instead of "Paradise".
event_counts = []

start = time.time()

for idx in tqdm(range(len(df))):
    row = df.iloc[idx]
    city_name = row['Principal City Name']
    state = row['State'].strip()
    
    # Since the principal city name (from Census data) is used, some city names might need to be cleaned up before querying the database.
    if city_name == "Indianapolis city (balance)" and state == "IN":
        city_name = "Indianapolis"
        
    if city_name == "San Buenaventura (Ventura)" and state == "CA":
        city_name = "Ventura"
        
    if city_name == "Urban Honolulu" and state == "HI":
        city_name = "Honolulu"
    
    if city_name == "Boise City" and state == "ID":
        city_name = "Boise"
    
    q = f"""
    SELECT count(*)
       FROM venue
       INNER JOIN event on event.venue_id = venue.id
       INNER JOIN artist_to_event on event.id = artist_to_event.event_id
       INNER JOIN artist on artist.id = artist_to_event.artist_id
       where 
         start_time >= '{start_date}' and start_time <= '{end_date}'
         and ticket_url not like '%ithaca.com%'
         and venue.city_name like "{city_name}"
         and venue.state like '{state}'
       ORDER BY event.start_time;
       """
    
    # The 1 event listed in Paradise, NV is a typo in the database - it was listed in "aradise" instead of "Paradise".
    num_events = int(pd.read_sql_query(q, conn).iloc[0]) if not (city_name == "Paradise" and state.strip() == "NV") else 1 
    
    event_counts.append(num_events)

    
print('Time elapsed:', time.time() - start, 'seconds')

df['Number of Events'] = event_counts
df['Live Event Music Rate (LMER)'] = df['Number of Events'] / df['Population']

100%|██████████| 1246/1246 [00:14<00:00, 86.54it/s] 

Time elapsed: 14.400950193405151 seconds


In [133]:
# Close connection
conn.close()

### Finally, Write to CSV File!

In [5]:
# Write to file!
df = df.sort_values('Live Event Music Rate (LMER)', ascending=False)
df.to_csv('data/LocalifyMusicEvents-USA-2019_full.csv', index=False)

### Apply Filter for Paper: Population must have 10K+ people in each city!

Write this to a separate dataset file. 

In [6]:
# df = pd.read_csv('data/LocalifyMusicEvents-USA-2019_full.csv')
df_paper = df[df['Population'] >= 10000]
df_paper.to_csv('data/LocalifyMusicEvents-USA-2019_paper.csv', index=False)

### Note:

For cells that involve scraping web pages (and take a long time to run), if you see an error that says "list out of range", try waiting for a while, then run it again. Often times, you might have had too many requests in too short period of time, which often times links to a CAPTCHA page. 